In [13]:
import os
import requests
from tqdm import tqdm
import zipfile

def download_file_with_progress(url, destination):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kibibyte
    with open(destination, 'wb') as file, tqdm(
        desc=destination,
        total=total_size,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in response.iter_content(block_size):
            file.write(data)
            bar.update(len(data))

# Define the URL and destination paths


glove_url = "https://nlp.stanford.edu/data/glove.840B.300d.zip"
glove_zip_path = "glove.840B.zip"
glove_txt_file = "glove.840B.300d.txt"


# glove_url = "http://nlp.stanford.edu/data/glove.6B.zip"
# glove_zip_path = "glove.6B.zip"
# glove_txt_file = "glove.6B.50d.txt"

# Download the GloVe zip file with progress bar if it doesn't exist
if not os.path.exists(glove_zip_path):
    download_file_with_progress(glove_url, glove_zip_path)

# Extract the required GloVe file if it doesn't exist
if not os.path.exists(glove_txt_file):
    with zipfile.ZipFile(glove_zip_path, 'r') as zip_ref:
        zip_ref.extract(glove_txt_file)


glove.840B.zip: 100%|██████████| 2.03G/2.03G [06:49<00:00, 5.31MiB/s]


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Configuration
CFG = {
    'embedding_dim': 300,
    'hidden_dim': 256,
    'batch_size': 2048,
    'num_epochs': 50,
    'learning_rate': 3e-4,
    'num_workers': 4,
    'glove_path': 'glove.840B.300d.txt'
}

# Dataset class
class IMDBDataset(Dataset):
    def __init__(self, texts, labels, word2idx):
        self.texts = [self.encode_text(text, word2idx) for text in texts]
        self.labels = labels

    def encode_text(self, text, word2idx):
        return torch.tensor([word2idx.get(word, word2idx['<unk>']) for word in text], dtype=torch.long)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Collate function for DataLoader
def collate_fn(batch):
    texts, labels = zip(*batch)
    lengths = torch.tensor([len(t) for t in texts])
    padded = pad_sequence(texts, batch_first=True, padding_value=0)
    return padded, lengths, torch.tensor(labels)

# Model architecture
class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_matrix):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.lstm = nn.LSTM(
            input_size=CFG['embedding_dim'],
            hidden_size=CFG['hidden_dim'],
            num_layers=10,
            bidirectional=True,
            dropout=0.3,
            batch_first=True
        )
        self.classifier = nn.Sequential(
            nn.Linear(CFG['hidden_dim']*2, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2)
        )

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = pack_padded_sequence(
            embedded, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        output, (hidden, cell) = self.lstm(packed)
        hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)
        return self.classifier(hidden)

# Load GloVe embeddings
def load_glove_embeddings(glove_path, word2idx):
    embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), CFG['embedding_dim']))
    with open(glove_path, 'r', encoding='utf-8') as f:
        for line in tqdm(f, desc='Loading GloVe'):
            parts = line.strip().split()
            word = parts[0]
            vector = parts[1:]
            if word in word2idx:
                try:
                    embeddings[word2idx[word]] = np.array(vector, dtype=np.float32)
                except ValueError:
                    print(f"Skipping malformed line: {line[:50]}...")
    return torch.tensor(embeddings, dtype=torch.float32)


# Training function
def train_model():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load and prepare data
    imdb_data = pd.read_csv('https://media.githubusercontent.com/media/EXC3ll3NTrhyTHM/NLP_datasets/refs/heads/main/IMDB_datasets/IMDB_cleaned.csv')
    texts = imdb_data['oov_processed_text'].astype(str).str.lower().str.split().tolist()
    labels = imdb_data['label'].astype(int).tolist()

    # Build vocabulary
    word_counts = {}
    for sentence in texts:
        for word in sentence:
            word_counts[word] = word_counts.get(word, 0) + 1
    word2idx = {'<pad>': 0, '<unk>': 1}
    for word in word_counts:
        if word_counts[word] > 5:
            word2idx[word] = len(word2idx)

    # Prepare datasets
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        texts, labels, test_size=0.2, random_state=42
    )

    train_dataset = IMDBDataset(train_texts, train_labels, word2idx)
    val_dataset = IMDBDataset(val_texts, val_labels, word2idx)

    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG['batch_size'],
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=CFG['num_workers'],
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=CFG['batch_size'],
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=CFG['num_workers'],
        pin_memory=True
    )

    # Initialize model
    embedding_matrix = load_glove_embeddings(CFG['glove_path'], word2idx)
    # model = SentimentLSTM(len(word2idx), embedding_matrix).to(device)
    
    model = SentimentLSTM(len(word2idx), embedding_matrix)
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs!")
        model = nn.DataParallel(model)
    model = model.to(device)


    optimizer = torch.optim.AdamW(model.parameters(), lr=CFG['learning_rate'])
    criterion = nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(CFG['num_epochs']):
        model.train()
        total_loss = 0
        for inputs, lengths, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}'):
            inputs, lengths, labels = inputs.to(device), lengths.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs, lengths)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch+1} | Training Loss: {avg_loss:.4f}')

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, lengths, labels in val_loader:
                inputs, lengths, labels = inputs.to(device), lengths.to(device), labels.to(device)
                outputs = model(inputs, lengths)
                preds = torch.argmax(outputs, dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        accuracy = correct / total
        print(f'Epoch {epoch+1} | Validation Accuracy: {accuracy:.2%}')
        
    return model, val_dataset

# Execute training
model, val_dataset = train_model()


Loading GloVe: 547252it [00:04, 112819.04it/s]

Skipping malformed line: contact name@domain.com 0.016426 0.13728 0.18781 0...


Loading GloVe: 2196017it [00:19, 113211.37it/s]


Using 4 GPUs!


Epoch 1: 100%|██████████| 20/20 [00:06<00:00,  3.12it/s]

...

Epoch 49 | Training Loss: 0.1045


Epoch 49 | Validation Accuracy: 87.58%


Epoch 50: 100%|██████████| 20/20 [00:05<00:00,  3.48it/s]

Epoch 50 | Training Loss: 0.0966


Epoch 50 | Validation Accuracy: 87.77%
